In [ ]:
#se recomienda instalar las siguientes librerías
# pip install pandas
# pip install tensorflow
# pip install matplotlib
# pip install optuna
# pip install scikit-learn
# pip install keras
# pip install tensorflow
# pip install scipy
# pip install python-pptx
#IMPORTACION DE LAS LIBRERIAS A UTILIZAR

import pandas as pd #librería para analizar datos
import numpy as np #librería para algebra linear
import tensorflow as tf #librería para modelos de deep learning
import matplotlib.pyplot as plt # libreraía para realizar los graficos
import optuna # libreria para optimizar modelo
from sklearn.model_selection import train_test_split #parte de la librería que divide los datos
from sklearn.metrics import roc_curve, auc, roc_auc_score # libreria para calcular las metricas en la curva ROC
from sklearn.metrics import precision_score, recall_score, f1_score # librerias para calcular la precision, el recall y el f1 score
from sklearn.preprocessing import MinMaxScaler #libreria para estandarizacion de datos
from sklearn.preprocessing import StandardScaler #libreria para estandarizacion de datos 
from keras.models import Sequential #importar el inicio de un modelo secuencial
from keras.layers import LSTM, Dense # importa capas densas y de LSTM
from sklearn.metrics import confusion_matrix #parte de la librería para hacer la matriz de confusion
from tensorflow import keras #importa keras que se encarga de la transformacion de los datos
from spicy import stats
import os
from tensorflow.keras.models import load_model
import os
from pptx import Presentation
from io import BytesIO
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [ ]:
# bootstrapping 
numero = 10 #numero de bootstrapping
data5 = pd.DataFrame()
ruut = r'C:\Users\colazabal\Desktop\Nueva carpeta (2)' #carpeta donde se encuentran los excels
g = 0

# Ruta donde se guardarán las diapositivas
ruta_presentacion = r"C:\Users\colazabal\Desktop\Nueva carpeta (2)" # donde guardaremos la ppt
# Crear una presentación vacía
ppt = Presentation()

#LOOP PARA CADA EXCEL QUE ESTÉ EN LA RUTA
for x in os.listdir(ruut):

    if x.endswith('xlsx'):
        root = os.path.join(ruut, x)
        data = pd.read_excel(root)

        for y in data.columns[1:]:
            if len(data[y].dropna()) < 70:
                g = g + 1

                data1 = data[y].dropna().diff()
                data1 = data1[1:]
                replica = []
                print(len(data1))
                for dato in range(numero):
                    bootstrap_replica = np.random.choice(data1, len(data1),replace=True)
                    replica.append(bootstrap_replica)

                df = pd.DataFrame({f'{y}': replica})
                df = df.explode(f'{y}').reset_index(drop=True)
                data1 = pd.DataFrame(data1)
                data2 = pd.concat([data1,df], axis=0).reset_index(drop=True)
                primer_valor = data[y].dropna().reset_index(drop=True)[0]
                data2['acum'] = data2[f'{y}'].cumsum()
                data2['serie acum'] = primer_valor + data2['acum']
                data5[y] = data2['serie acum']
                
                try: 
                    empresa = y

                    valor1 = len(data5[empresa].dropna())
                    valor1 = int(valor1) #longitud del la parte de entrenamiento
                    TIME_STEPS = 30 #valor que se puede variar en el tiempo - CONSULTAR A MARVIN SI SE PUEDE REDUCIR EL NUMERO PARA SERIES PEQUEÑAS.
                    epochs = 100 
                    batch_size = 32

                    #PREPROCESAMIENTO DE LOS DATOS

                    #PREPROCESAMIENTO DE LOS DATOS

                    data5['valor_normalizado'] = data5[empresa].dropna().reset_index(drop = True)
                    scaler = StandardScaler()
                    scaler = scaler.fit(data5[['valor_normalizado']])
                    data5['valor_normalizado'] = scaler.transform(data5[['valor_normalizado']])

                    def create_dataset(X, y, time_steps=1):
                        Xs, ys = [], []
                        for i in range(len(X) - time_steps):
                            v = X.iloc[i:(i + time_steps)].values
                            Xs.append(v)        
                            ys.append(y.iloc[i + time_steps])
                        return np.array(Xs), np.array(ys)
                    


                    X_data, y_data = create_dataset(data5[['valor_normalizado']], data5.valor_normalizado, TIME_STEPS)

                    #CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
                    model = tf.keras.Sequential([
                        tf.keras.layers.Input(shape=(X_data.shape[1])),
                        tf.keras.layers.Dense(64, activation='relu'),
                        tf.keras.layers.Dense(32, activation='relu'),
                        tf.keras.layers.Dense(16, activation='relu'),
                        tf.keras.layers.Dense(32, activation='relu'),
                        tf.keras.layers.Dense(64, activation='relu'),
                        tf.keras.layers.Dense(X_data.shape[1], activation='linear')])

                    model.compile(optimizer='adam', loss='mean_squared_error')
                    history = model.fit(X_data, X_data, epochs=epochs, batch_size=batch_size, validation_data=(X_data, X_data), verbose=0) #ENTRENAMIENTO
                    mse = model.evaluate(X_data, X_data, verbose=0)
                    loss = history.history['loss']
                    val_loss = history.history['val_loss']

                    #predicciones
                    predictions = model.predict(X_data)
                    predicciones1 = pd.DataFrame(predictions.flat[0:TIME_STEPS])

                    predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
                    predicciones = pd.concat((predicciones1, predicciones2),axis=0)

                    predicciones.reset_index(drop=True, inplace=True)
                    
                    #calculo del error de reconstrucción
                    df_test = pd.DataFrame()
                    df_test['x_test'] = data5['valor_normalizado']
                    df_test['predic'] = predicciones
                    df_test['error'] = df_test['x_test'] - df_test['predic']
                    df_test['error2'] = df_test['error']**2

                    #cálculo del threshold
                    mu = np.mean(df_test['error2'])  # Media
                    sigma = np.std(df_test['error2'])  # Desviación estándar
                    alpha = 0.05  # Nivel de significancia del 5%
                    z_score = stats.norm.ppf(1 - alpha/2)  # Utilizando la función de percentil inverso de la distribución normal
                    threshold = mu - (z_score * (sigma / np.sqrt(len(df_test['error2'])))) #Umbral gaussiano

                    anomalias = df_test[df_test['error2'] > threshold] #creación de la variable anomalias que contiene a aquellos errores mayores al umbral


                    def objective(trial):
                        # Definir los espacios de búsqueda para los hiperparámetros
                        num_units1 = trial.suggest_int('num_units1', 64, 128)
                        num_units2 = trial.suggest_int('num_units2', 32, 64)
                        num_units3 = trial.suggest_int('num_units3', 8, 16)
                        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
                        # Crear un modelo Autoencoder
                        model = tf.keras.Sequential([
                            tf.keras.layers.Input(shape=(X_data.shape[1])),
                            tf.keras.layers.Dense(num_units1, activation='relu'),
                            tf.keras.layers.Dense(num_units2, activation='relu'),
                            tf.keras.layers.Dense(num_units3, activation='relu'),
                            tf.keras.layers.Dense(num_units2, activation='relu'),
                            tf.keras.layers.Dense(num_units1, activation='relu'),
                            tf.keras.layers.Dense(X_data.shape[1], activation='linear')])
                        # Compilar el modelo con los hiperparámetros seleccionados
                        model.compile(optimizer='adam', loss='mean_squared_error')  # Corregido

                        # Entrenar el modelo con los datos de entrenamiento y prueba
                        history = model.fit(X_data, X_data, epochs=100, batch_size=batch_size, validation_data=(X_data, X_data), verbose=0)

                        # Calcular la métrica de interés (puede ser la precisión, el AUC, etc.)
                        # En este ejemplo, usaremos la pérdida en los datos de prueba
                        loss = model.evaluate(X_data, X_data)  # Corregido

                        return loss

                    # Crear el estudio de Optuna y ejecutar la minimizacion de la pérdida
                    study = optuna.create_study(direction='minimize')
                    study.optimize(objective, n_trials=10)

                    # Obtener los mejores hiperparámetros encontrados
                    best_params = study.best_params

                    # Imprimir los mejores hiperparámetros
                    print("Mejores hiperparámetros encontrados:", best_params)

                    #UTILIZAMOS EL MODELO CON LOS NUEVOS PARÁMETROS, Y BATCHSIZE
                    #CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
                    model = tf.keras.Sequential([
                        tf.keras.layers.Input(shape=(X_data.shape[1])),
                        tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
                        tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
                        tf.keras.layers.Dense(best_params['num_units3'], activation='relu'),
                        tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
                        tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
                        tf.keras.layers.Dense(X_data.shape[1], activation='linear')])

                    model.compile(optimizer='adam', loss='mean_squared_error')
                    history = model.fit(X_data, X_data, epochs=epochs, batch_size=best_params['batch_size'], validation_data=(X_data, X_data), verbose=0)
                    mse = model.evaluate(X_data, X_data, verbose=0)


                    predictions = model.predict(X_data)
                    predicciones1 = pd.DataFrame(predictions[0].flat)
                    predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
                    predicciones = pd.concat((predicciones1, predicciones2))

                    predicciones.reset_index(drop=True, inplace=True)

                    loss = history.history['loss']
                    val_loss = history.history['val_loss']

                    df_test = pd.DataFrame()
                    df_test['x_test'] = data5['valor_normalizado']
                    df_test['predic'] = predicciones
                    df_test['error'] = df_test['x_test'] - df_test['predic']
                    df_test['error2'] = df_test['error']**2

                    df_test = df_test.iloc[:len(data1)]
                    anomalias = df_test[df_test['error2'] > threshold] #ANOMALIA DE ACUERDO A LO QUE SEA MAYOR AL THRESHOLD
                    
                    # Crear una nueva figura para el segundo gráfico

                    plt.figure(figsize=(12, 6))
                    plt.plot(data5['valor_normalizado'][:len(data1)], label='Datos Originales', color='blue')
                    plt.title(f'{empresa}')
                    plt.xlabel('Índice de Datos')
                    plt.ylabel('Valor Normalizado')
                    plt.legend()
                    plt.grid(True)
                    buf2 = BytesIO()
                    plt.savefig(buf2, format='png')
                    plt.show()
                    plt.close()
                    left1, top1 = 0, 0  # Ajusta la posición de la primera imagen
                    slide = ppt.slides.add_slide(ppt.slide_layouts[5])
                    pic2 = slide.shapes.add_picture(buf2, left1, top1, width=ppt.slide_width, height=ppt.slide_height)
                    
                    # Crear una figura y un solo eje y
                    fig, ax = plt.subplots(figsize=(12, 6))
                    # Gráfico de Datos Originales y Predicciones
                    ax.plot(data5['valor_normalizado'].iloc[:len(data1)], label='Datos Originales', color='blue', linestyle = '--', linewidth = 0.8)
                    ax.plot(predicciones.iloc[:len(data1)], label='Predicciones', color='black')
                    ax.scatter(anomalias.index[:len(data1)], anomalias['x_test'].iloc[:len(data1)], color='red', label='Anomalías', marker='o', s=20)
                    ax.set_title('Datos de tráfico vs. Predicciones (con Anomalías)', fontweight = 'bold')
                    ax.set_xlabel('Datos de Testeo')
                    ax.set_ylabel('Valor Normalizado', color='black')  # Eje izquierdo
                    ax.legend(loc='upper left')

                    # Crear un segundo conjunto de etiquetas en el eje y para la pérdida y el umbral
                    ax2 = ax.twinx()
                    ax2.plot(df_test.index[:len(data1)], df_test.error2.iloc[:len(data1)], label='Pérdida', color='red', linestyle='--', linewidth = 1.5)
                    ax2.set_ylabel('Pérdida y Umbral', color='black')  # Eje derecho
                    ax2.legend(loc='upper right')
                    y2_min = 0
                    y2_max = max(df_test.error2)*4

                    # Establecer los límites en el eje y del segundo conjunto de datos
                    ax2.set_ylim(y2_min, y2_max)  # Reemplaza y2_min y y2_max con los valores deseados

                    # Ajustar el espaciado entre los gráficos
                    plt.tight_layout()
                    buf = BytesIO()
                    plt.savefig(buf, format='png')
                    # Mostrar el gráfico combinado
                    plt.show()
                    # Crear una nueva diapositiva
                    slide = ppt.slides.add_slide(ppt.slide_layouts[5])  # 5 es el diseño de título y contenido

                    # Insertar la primera imagen del gráfico
                    left1, top1 = 0, 0  # Ajusta la posición de la primera imagen
                    pic1 = slide.shapes.add_picture(buf, left1, top1, width=ppt.slide_width, height=ppt.slide_height)
                except:
                    print(f'error con el archivo {x}')

In [ ]:
#se recomienda instalar las siguientes librerías
# pip install pandas
# pip install tensorflow
# pip install matplotlib
# pip install optuna
# pip install scikit-learn
# pip install keras
# pip install tensorflow
# pip install scipy
# pip install python-pptx

#IMPORTACION DE LAS LIBRERIAS A UTILIZAR

import pandas as pd #librería para analizar datos
import numpy as np #librería para algebra linear
import tensorflow as tf #librería para modelos de deep learning
import matplotlib.pyplot as plt # libreraía para realizar los graficos
import optuna # libreria para optimizar modelo
from sklearn.model_selection import train_test_split #parte de la librería que divide los datos
from sklearn.metrics import roc_curve, auc, roc_auc_score # libreria para calcular las metricas en la curva ROC
from sklearn.metrics import precision_score, recall_score, f1_score # librerias para calcular la precision, el recall y el f1 score
from sklearn.preprocessing import MinMaxScaler #libreria para estandarizacion de datos
from sklearn.preprocessing import StandardScaler #libreria para estandarizacion de datos 
from keras.models import Sequential #importar el inicio de un modelo secuencial
from keras.layers import LSTM, Dense # importa capas densas y de LSTM
from sklearn.metrics import confusion_matrix #parte de la librería para hacer la matriz de confusion
from tensorflow import keras #importa keras que se encarga de la transformacion de los datos
from spicy import stats
import os
from tensorflow.keras.models import load_model
import os
from pptx import Presentation
from io import BytesIO
from sklearn.preprocessing import StandardScaler
from scipy import stats

# series normales 
numero = 10 #cantidad de replicas de los datos
data5 = pd.DataFrame()
ruut = r'\\srvfiledfs\documentos\DPRC\2023\FINANZAS\COLAZABAL\14. Recurrent Neural Network\Nueva carpeta' #carpeta donde se encuentran los excels
g = 0
# Ruta donde se guardarán las diapositivas
ruta_presentacion = r"C:\Users\colazabal\Desktop\Nueva carpeta\presentacion" # donde guardaremos la ppt
# Crear una presentación vacía
ppt = Presentation()

#en estos casos no se realiza bootstrap solo se 

for x in os.listdir(ruut):
    if x.endswith('xlsx'):
        root = os.path.join(ruut, x)
        data = pd.read_excel(root)
        for y in data.columns[1:]:             
            try: 
                empresa = y

                valor1 = len(data5[empresa].dropna())
                valor1 = int(valor1) #longitud del la parte de entrenamiento
                TIME_STEPS = 30 #valor que se puede variar en el tiempo
                epochs = 100 
                batch_size = 32 

                #PREPROCESAMIENTO DE LOS DATOS

                data5['valor_normalizado'] = data5[empresa].dropna().reset_index(drop = True)
                scaler = StandardScaler()
                scaler = scaler.fit(data5[['valor_normalizado']])
                data5['valor_normalizado'] = scaler.transform(data5[['valor_normalizado']])

                def create_dataset(X, y, time_steps=1):
                    Xs, ys = [], []
                    for i in range(len(X) - time_steps):
                        v = X.iloc[i:(i + time_steps)].values
                        Xs.append(v)        
                        ys.append(y.iloc[i + time_steps])
                    return np.array(Xs), np.array(ys)

                X_data, y_data = create_dataset(data5[['valor_normalizado']], data5.valor_normalizado, TIME_STEPS)

                #CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
                model = tf.keras.Sequential([
                    tf.keras.layers.Input(shape=(X_data.shape[1])),
                    tf.keras.layers.Dense(64, activation='relu'),
                    tf.keras.layers.Dense(32, activation='relu'),
                    tf.keras.layers.Dense(16, activation='relu'),
                    tf.keras.layers.Dense(32, activation='relu'),
                    tf.keras.layers.Dense(64, activation='relu'),
                    tf.keras.layers.Dense(X_data.shape[1], activation='linear')])

                model.compile(optimizer='adam', loss='mean_squared_error')
                history = model.fit(X_data, X_data, epochs=epochs, batch_size=batch_size, validation_data=(X_data, X_data), verbose=0) #ENTRENAMIENTO
                mse = model.evaluate(X_data, X_data, verbose=0)
                loss = history.history['loss']
                val_loss = history.history['val_loss']

                #predicciones
                predictions = model.predict(X_data)
                predicciones1 = pd.DataFrame(predictions.flat[0:TIME_STEPS])

                predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
                predicciones = pd.concat((predicciones1, predicciones2),axis=0)

                predicciones.reset_index(drop=True, inplace=True)
                
                #calculo del error de reconstrucción
                df_test = pd.DataFrame()
                df_test['x_test'] = data5['valor_normalizado']
                df_test['predic'] = predicciones
                df_test['error'] = df_test['x_test'] - df_test['predic']
                df_test['error2'] = df_test['error']**2

                #cálculo del threshold
                mu = np.mean(df_test['error2'])  # Media
                sigma = np.std(df_test['error2'])  # Desviación estándar
                alpha = 0.05  # Nivel de significancia del 5%
                z_score = stats.norm.ppf(1 - alpha/2)  # Utilizando la función de percentil inverso de la distribución normal
                threshold = mu - (z_score * (sigma / np.sqrt(len(df_test['error2'])))) #Umbral gaussiano

                anomalias = df_test[df_test['error2'] > threshold] #creación de la variable anomalias que contiene a aquellos errores mayores al umbral


                def objective(trial):
                    # Definir los espacios de búsqueda para los hiperparámetros
                    num_units1 = trial.suggest_int('num_units1', 64, 128)
                    num_units2 = trial.suggest_int('num_units2', 32, 64)
                    num_units3 = trial.suggest_int('num_units3', 8, 16)
                    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
                    # Crear un modelo Autoencoder
                    model = tf.keras.Sequential([
                        tf.keras.layers.Input(shape=(X_data.shape[1])),
                        tf.keras.layers.Dense(num_units1, activation='relu'),
                        tf.keras.layers.Dense(num_units2, activation='relu'),
                        tf.keras.layers.Dense(num_units3, activation='relu'),
                        tf.keras.layers.Dense(num_units2, activation='relu'),
                        tf.keras.layers.Dense(num_units1, activation='relu'),
                        tf.keras.layers.Dense(X_data.shape[1], activation='linear')])
                    # Compilar el modelo con los hiperparámetros seleccionados
                    model.compile(optimizer='adam', loss='mean_squared_error')  # Corregido

                    # Entrenar el modelo con los datos de entrenamiento y prueba
                    history = model.fit(X_data, X_data, epochs=100, batch_size=batch_size, validation_data=(X_data, X_data), verbose=0)

                    # Calcular la métrica de interés (puede ser la precisión, el AUC, etc.)
                    # En este ejemplo, usaremos la pérdida en los datos de prueba
                    loss = model.evaluate(X_data, X_data)  # Corregido

                    return loss

                # Crear el estudio de Optuna y ejecutar la minimizacion de la pérdida
                study = optuna.create_study(direction='minimize')
                study.optimize(objective, n_trials=10)

                # Obtener los mejores hiperparámetros encontrados
                best_params = study.best_params

                # Imprimir los mejores hiperparámetros
                print("Mejores hiperparámetros encontrados:", best_params)

                #UTILIZAMOS EL MODELO CON LOS NUEVOS PARÁMETROS, Y BATCHSIZE
                #CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
                model = tf.keras.Sequential([
                    tf.keras.layers.Input(shape=(X_data.shape[1])),
                    tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
                    tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
                    tf.keras.layers.Dense(best_params['num_units3'], activation='relu'),
                    tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
                    tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
                    tf.keras.layers.Dense(X_data.shape[1], activation='linear')])

                model.compile(optimizer='adam', loss='mean_squared_error')
                history = model.fit(X_data, X_data, epochs=epochs, batch_size=best_params['batch_size'], validation_data=(X_data, X_data), verbose=0)
                mse = model.evaluate(X_data, X_data, verbose=0)


                predictions = model.predict(X_data)
                predicciones1 = pd.DataFrame(predictions[0].flat)
                predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
                predicciones = pd.concat((predicciones1, predicciones2))

                predicciones.reset_index(drop=True, inplace=True)

                loss = history.history['loss']
                val_loss = history.history['val_loss']

                df_test = pd.DataFrame()
                df_test['x_test'] = data5['valor_normalizado']
                df_test['predic'] = predicciones
                df_test['error'] = df_test['x_test'] - df_test['predic']
                df_test['error2'] = df_test['error']**2

                df_test = df_test.iloc[:len(data1)]
                anomalias = df_test[df_test['error2'] > threshold] #ANOMALIA DE ACUERDO A LO QUE SEA MAYOR AL THRESHOLD
                
                # Crear una nueva figura para el segundo gráfico

                plt.figure(figsize=(12, 6))
                plt.plot(data5['valor_normalizado'][:len(data1)], label='Datos Originales', color='blue')
                plt.title(f'{empresa}')
                plt.xlabel('Índice de Datos')
                plt.ylabel('Valor Normalizado')
                plt.legend()
                plt.grid(True)
                buf2 = BytesIO()
                plt.savefig(buf2, format='png')
                plt.show()
                plt.close()
                left1, top1 = 0, 0  # Ajusta la posición de la primera imagen
                slide = ppt.slides.add_slide(ppt.slide_layouts[5])
                pic2 = slide.shapes.add_picture(buf2, left1, top1, width=ppt.slide_width, height=ppt.slide_height)
                
                # Crear una figura y un solo eje y
                fig, ax = plt.subplots(figsize=(12, 6))
                # Gráfico de Datos Originales y Predicciones
                ax.plot(data5['valor_normalizado'].iloc[:len(data1)], label='Datos Originales', color='blue', linestyle = '--', linewidth = 0.8)
                ax.plot(predicciones.iloc[:len(data1)], label='Predicciones', color='black')
                ax.scatter(anomalias.index[:len(data1)], anomalias['x_test'].iloc[:len(data1)], color='red', label='Anomalías', marker='o', s=20)
                ax.set_title('Datos de tráfico vs. Predicciones (con Anomalías)', fontweight = 'bold')
                ax.set_xlabel('Datos de Testeo')
                ax.set_ylabel('Valor Normalizado', color='black')  # Eje izquierdo
                ax.legend(loc='upper left')

                # Crear un segundo conjunto de etiquetas en el eje y para la pérdida y el umbral
                ax2 = ax.twinx()
                ax2.plot(df_test.index[:len(data1)], df_test.error2.iloc[:len(data1)], label='Pérdida', color='red', linestyle='--', linewidth = 1.5)
                ax2.set_ylabel('Pérdida y Umbral', color='black')  # Eje derecho
                ax2.legend(loc='upper right')
                y2_min = 0
                y2_max = max(df_test.error2)*4

                # Establecer los límites en el eje y del segundo conjunto de datos
                ax2.set_ylim(y2_min, y2_max)  # Reemplaza y2_min y y2_max con los valores deseados

                # Ajustar el espaciado entre los gráficos
                plt.tight_layout()
                buf = BytesIO()
                plt.savefig(buf, format='png')
                # Mostrar el gráfico combinado
                plt.show()
                # Crear una nueva diapositiva
                slide = ppt.slides.add_slide(ppt.slide_layouts[5])  # 5 es el diseño de título y contenido

                # Insertar la primera imagen del gráfico
                left1, top1 = 0, 0  # Ajusta la posición de la primera imagen
                pic1 = slide.shapes.add_picture(buf, left1, top1, width=ppt.slide_width, height=ppt.slide_height)
            except:
                print(f'error con el archivo {x}')

In [ ]:
ppt.save(r'C:\Users\colazabal\Desktop\Nueva carpeta\pptnuevo4.pptx')

In [ ]:
ruta2 = r'C:\Users\colazabal\Desktop\Nueva carpeta\presentacion'
# Ruta donde se guardarán las diapositivas
ruta_presentacion = r"C:\Users\colazabal\Desktop\Nueva carpeta\presentacion"
# Crear una presentación vacía
ppt = Presentation()

for x in data5.columns:
        try: 
            empresa = x

            valor1 = len(data5[empresa].dropna())
            valor1 = int(valor1) #longitud del la parte de entrenamiento
            TIME_STEPS = 30 #valor que se puede variar en el tiempo
            epochs = 100 
            batch_size = 32

            #PREPROCESAMIENTO DE LOS DATOS

            #PREPROCESAMIENTO DE LOS DATOS

            data5['valor_normalizado'] = data5[empresa].dropna().reset_index(drop = True)
            scaler = StandardScaler()
            scaler = scaler.fit(data5[['valor_normalizado']])
            data5['valor_normalizado'] = scaler.transform(data5[['valor_normalizado']])

            def create_dataset(X, y, time_steps=1):
                Xs, ys = [], []
                for i in range(len(X) - time_steps):
                    v = X.iloc[i:(i + time_steps)].values
                    Xs.append(v)        
                    ys.append(y.iloc[i + time_steps])
                return np.array(Xs), np.array(ys)
            


            X_data, y_data = create_dataset(data5[['valor_normalizado']], data5.valor_normalizado, TIME_STEPS)

            #CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
            model = tf.keras.Sequential([
                tf.keras.layers.Input(shape=(X_data.shape[1])),
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(32, activation='relu'),
                tf.keras.layers.Dense(16, activation='relu'),
                tf.keras.layers.Dense(32, activation='relu'),
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(X_data.shape[1], activation='linear')])

            model.compile(optimizer='adam', loss='mean_squared_error')
            history = model.fit(X_data, X_data, epochs=epochs, batch_size=batch_size, validation_data=(X_data, X_data), verbose=0) #ENTRENAMIENTO
            mse = model.evaluate(X_data, X_data, verbose=0)
            loss = history.history['loss']
            val_loss = history.history['val_loss']

            #predicciones
            predictions = model.predict(X_data)
            predicciones1 = pd.DataFrame(predictions.flat[0:TIME_STEPS])

            predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
            predicciones = pd.concat((predicciones1, predicciones2),axis=0)

            predicciones.reset_index(drop=True, inplace=True)
            
            #calculo del error de reconstrucción
            df_test = pd.DataFrame()
            df_test['x_test'] = data5['valor_normalizado']
            df_test['predic'] = predicciones
            df_test['error'] = df_test['x_test'] - df_test['predic']
            df_test['error2'] = df_test['error']**2

            #cálculo del threshold
            mu = np.mean(df_test['error2'])  # Media
            sigma = np.std(df_test['error2'])  # Desviación estándar
            alpha = 0.05  # Nivel de significancia del 5%
            z_score = stats.norm.ppf(1 - alpha/2)  # Utilizando la función de percentil inverso de la distribución normal
            threshold = mu - (z_score * (sigma / np.sqrt(len(df_test['error2'])))) #Umbral gaussiano

            anomalias = df_test[df_test['error2'] > threshold] #creación de la variable anomalias que contiene a aquellos errores mayores al umbral


            def objective(trial):
                # Definir los espacios de búsqueda para los hiperparámetros
                num_units1 = trial.suggest_int('num_units1', 64, 128)
                num_units2 = trial.suggest_int('num_units2', 32, 64)
                num_units3 = trial.suggest_int('num_units3', 8, 16)
                batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
                # Crear un modelo Autoencoder
                model = tf.keras.Sequential([
                    tf.keras.layers.Input(shape=(X_data.shape[1])),
                    tf.keras.layers.Dense(num_units1, activation='relu'),
                    tf.keras.layers.Dense(num_units2, activation='relu'),
                    tf.keras.layers.Dense(num_units3, activation='relu'),
                    tf.keras.layers.Dense(num_units2, activation='relu'),
                    tf.keras.layers.Dense(num_units1, activation='relu'),
                    tf.keras.layers.Dense(X_data.shape[1], activation='linear')])
                # Compilar el modelo con los hiperparámetros seleccionados
                model.compile(optimizer='adam', loss='mean_squared_error')  # Corregido

                # Entrenar el modelo con los datos de entrenamiento y prueba
                history = model.fit(X_data, X_data, epochs=100, batch_size=batch_size, validation_data=(X_data, X_data), verbose=0)

                # Calcular la métrica de interés (puede ser la precisión, el AUC, etc.)
                # En este ejemplo, usaremos la pérdida en los datos de prueba
                loss = model.evaluate(X_data, X_data)  # Corregido

                return loss

            # Crear el estudio de Optuna y ejecutar la minimizacion de la pérdida
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=2)

            # Obtener los mejores hiperparámetros encontrados
            best_params = study.best_params

            # Imprimir los mejores hiperparámetros
            print("Mejores hiperparámetros encontrados:", best_params)

            #UTILIZAMOS EL MODELO CON LOS NUEVOS PARÁMETROS, Y BATCHSIZE
            #CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
            model = tf.keras.Sequential([
                tf.keras.layers.Input(shape=(X_data.shape[1])),
                tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
                tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
                tf.keras.layers.Dense(best_params['num_units3'], activation='relu'),
                tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
                tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
                tf.keras.layers.Dense(X_data.shape[1], activation='linear')])

            model.compile(optimizer='adam', loss='mean_squared_error')
            history = model.fit(X_data, X_data, epochs=epochs, batch_size=best_params['batch_size'], validation_data=(X_data, X_data), verbose=0)
            mse = model.evaluate(X_data, X_data, verbose=0)


            predictions = model.predict(X_data)
            predicciones1 = pd.DataFrame(predictions[0].flat)
            predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
            predicciones = pd.concat((predicciones1, predicciones2))

            predicciones.reset_index(drop=True, inplace=True)

            loss = history.history['loss']
            val_loss = history.history['val_loss']

            df_test = pd.DataFrame()
            df_test['x_test'] = data5['valor_normalizado']
            df_test['predic'] = predicciones
            df_test['error'] = df_test['x_test'] - df_test['predic']
            df_test['error2'] = df_test['error']**2

            anomalias = df_test[df_test['error2'] > threshold] #ANOMALIA DE ACUERDO A LO QUE SEA MAYOR AL THRESHOLD
            
            # Crear una nueva figura para el segundo gráfico

            plt.figure(figsize=(12, 6))
            plt.plot(data5['valor_normalizado'][:len(data5[empresa])], label='Datos Originales', color='blue')
            plt.title(f'{empresa}')
            plt.xlabel('Índice de Datos')
            plt.ylabel('Valor Normalizado')
            plt.legend()
            plt.grid(True)
            buf2 = BytesIO()
            plt.savefig(buf2, format='png')
            plt.show()
            plt.close()
            left1, top1 = 0, 0  # Ajusta la posición de la primera imagen
            slide = ppt.slides.add_slide(ppt.slide_layouts[5])
            pic2 = slide.shapes.add_picture(buf2, left1, top1, width=ppt.slide_width, height=ppt.slide_height)
            
            # Crear una figura y un solo eje y
            fig, ax = plt.subplots(figsize=(12, 6))
            # Gráfico de Datos Originales y Predicciones
            ax.plot(data5['valor_normalizado'][:len(data5[empresa])], label='Datos Originales', color='blue', linestyle = '--', linewidth = 0.8)
            ax.plot(predicciones[:len(data5[empresa])], label='Predicciones', color='black')
            ax.scatter(anomalias.index[:len(data5[empresa])], anomalias['x_test'][:len(data5[empresa])], color='red', label='Anomalías', marker='o', s=20)
            ax.set_title('Datos de tráfico vs. Predicciones (con Anomalías)', fontweight = 'bold')
            ax.set_xlabel('Datos de Testeo')
            ax.set_ylabel('Valor Normalizado', color='black')  # Eje izquierdo
            ax.legend(loc='upper left')

            # Crear un segundo conjunto de etiquetas en el eje y para la pérdida y el umbral
            ax2 = ax.twinx()
            ax2.plot(df_test.index[:len(data5[empresa])], df_test.error2[:len(data5[empresa])], label='Pérdida', color='red', linestyle='--', linewidth = 1.5)
            ax2.set_ylabel('Pérdida y Umbral', color='black')  # Eje derecho
            ax2.legend(loc='upper right')
            y2_min = 0
            y2_max = max(df_test.error2)*4

            # Establecer los límites en el eje y del segundo conjunto de datos
            ax2.set_ylim(y2_min, y2_max)  # Reemplaza y2_min y y2_max con los valores deseados

            # Ajustar el espaciado entre los gráficos
            plt.tight_layout()
            buf = BytesIO()
            plt.savefig(buf, format='png')
            # Mostrar el gráfico combinado
            plt.show()
            # Crear una nueva diapositiva
            slide = ppt.slides.add_slide(ppt.slide_layouts[5])  # 5 es el diseño de título y contenido

            # Insertar la primera imagen del gráfico
            left1, top1 = 0, 0  # Ajusta la posición de la primera imagen
            pic1 = slide.shapes.add_picture(buf, left1, top1, width=ppt.slide_width, height=ppt.slide_height)
        except:
            print(f'error con el archivo {x}')

In [ ]:
ppt.save(r'C:\Users\colazabal\Desktop\Nueva carpeta\pptnuevo3.pptx')

In [ ]:
ruta2 = r'C:\Users\colazabal\Desktop\Nueva carpeta\presentacion'
# Ruta donde se guardarán las diapositivas
ruta_presentacion = r"C:\Users\colazabal\Desktop\Nueva carpeta\presentacion"

# Crear una presentación vacía
ppt = Presentation()

for x in data5.columns:
        
    empresa = x


    test_size = 0.3 # longitud del testeo
    valor = 1 - test_size
    valor1 = valor*400
    valor1 = int(valor1) #longitud del la parte de entrenamiento
    TIME_STEPS = 30 #valor que se puede variar en el tiempo
    epochs = 100 
    batch_size = 32

    #PREPROCESAMIENTO DE LOS DATOS

    data['valor_normalizado'] = data5[empresa]
    train, test = train_test_split(data['valor_normalizado'], test_size=test_size, shuffle = False)
    train = pd.DataFrame(train)
    test= pd.DataFrame(test)
    scaler = StandardScaler()
    scaler = scaler.fit(train[['valor_normalizado']])
    train['valor_normalizado'] = scaler.transform(train[['valor_normalizado']])
    test['valor_normalizado'] = scaler.transform(test[['valor_normalizado']])

    def create_dataset(X, y, time_steps=1):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            v = X.iloc[i:(i + time_steps)].values
            Xs.append(v)        
            ys.append(y.iloc[i + time_steps])
        return np.array(Xs), np.array(ys)

    X_train, y_train = create_dataset(train[['valor_normalizado']], train.valor_normalizado, TIME_STEPS)
    X_test, y_test = create_dataset(test[['valor_normalizado']], test.valor_normalizado, TIME_STEPS)

    #CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1])),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(X_train.shape[1], activation='linear')])

    model.compile(optimizer='adam', loss='mean_squared_error')
    history = model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_data=(X_train, X_train), verbose=0) #ENTRENAMIENTO
    mse = model.evaluate(X_train, X_train, verbose=0)
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # desprocesamiento de los datos de testeo
    a = pd.DataFrame(X_test[0].flat)
    b = pd.DataFrame(y_test)
    datos_testeo = pd.concat((a,b)).reset_index(drop=True)

    #predicciones
    predictions = model.predict(X_test)
    predicciones1 = pd.DataFrame(predictions[0].flat)
    predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
    predicciones = pd.concat((predicciones1, predicciones2))
    predicciones.reset_index(drop=True, inplace=True)
    datos_testeo.reset_index(drop=True, inplace=True)

    #calculo del error de reconstrucción
    df_test = pd.DataFrame()
    df_test['x_test'] = datos_testeo
    df_test['predic'] = predicciones
    df_test['error'] = df_test['x_test'] - df_test['predic']
    df_test['error2'] = df_test['error']**2

    #cálculo del threshold
    mu = np.mean(df_test['error2'])  # Media
    sigma = np.std(df_test['error2'])  # Desviación estándar
    alpha = 0.05  # Nivel de significancia del 5%
    z_score = stats.norm.ppf(1 - alpha/2)  # Utilizando la función de percentil inverso de la distribución normal
    threshold = mu - (z_score * (sigma / np.sqrt(len(df_test['error2'])))) #Umbral gaussiano

    anomalias = df_test[df_test['error2'] > threshold] #creación de la variable anomalias que contiene a aquellos errores mayores al umbral


    def objective(trial):
        # Definir los espacios de búsqueda para los hiperparámetros
        num_units1 = trial.suggest_int('num_units1', 64, 128)
        num_units2 = trial.suggest_int('num_units2', 32, 64)
        num_units3 = trial.suggest_int('num_units3', 8, 16)
        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
        # Crear un modelo Autoencoder
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(X_train.shape[1])),
            tf.keras.layers.Dense(num_units1, activation='relu'),
            tf.keras.layers.Dense(num_units2, activation='relu'),
            tf.keras.layers.Dense(num_units3, activation='relu'),
            tf.keras.layers.Dense(num_units2, activation='relu'),
            tf.keras.layers.Dense(num_units1, activation='relu'),
            tf.keras.layers.Dense(X_train.shape[1], activation='linear')])
        # Compilar el modelo con los hiperparámetros seleccionados
        model.compile(optimizer='adam', loss='mean_squared_error')  # Corregido

        # Entrenar el modelo con los datos de entrenamiento y prueba
        history = model.fit(X_train, X_train, epochs=100, batch_size=batch_size, validation_data=(X_train, X_train), verbose=0)

        # Calcular la métrica de interés (puede ser la precisión, el AUC, etc.)
        # En este ejemplo, usaremos la pérdida en los datos de prueba
        loss = model.evaluate(X_train, X_train)  # Corregido

        return loss

    # Crear el estudio de Optuna y ejecutar la minimizacion de la pérdida
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)

    # Obtener los mejores hiperparámetros encontrados
    best_params = study.best_params

    # Imprimir los mejores hiperparámetros
    print("Mejores hiperparámetros encontrados:", best_params)

    #UTILIZAMOS EL MODELO CON LOS NUEVOS PARÁMETROS, Y BATCHSIZE
    #CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1])),
        tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
        tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
        tf.keras.layers.Dense(best_params['num_units3'], activation='relu'),
        tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
        tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
        tf.keras.layers.Dense(X_train.shape[1], activation='linear')])

    model.compile(optimizer='adam', loss='mean_squared_error')
    history = model.fit(X_train, X_train, epochs=50, batch_size=best_params['batch_size'], validation_data=(X_train, X_train), verbose=0)
    mse = model.evaluate(X_train, X_train, verbose=0)

    a = pd.DataFrame(X_test[0].flat)
    b = pd.DataFrame(y_test)
    datos_testeo = pd.concat((a,b)).reset_index(drop=True)

    predictions = model.predict(X_test)
    predicciones1 = pd.DataFrame(predictions[0].flat)
    predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
    predicciones = pd.concat((predicciones1, predicciones2))

    predicciones.reset_index(drop=True, inplace=True)
    datos_testeo.reset_index(drop=True, inplace=True)

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    df_test = pd.DataFrame()
    df_test['x_test'] = datos_testeo
    df_test['predic'] = predicciones
    df_test['error'] = df_test['x_test'] - df_test['predic']
    df_test['error2'] = df_test['error']**2

    anomalias = df_test[df_test['error2'] > threshold] #ANOMALIA DE ACUERDO A LO QUE SEA MAYOR AL THRESHOLD
    # Crear una nueva figura para el segundo gráfico
                
    plt.figure(figsize=(12, 6))
    plt.plot(data['valor_normalizado'], label='Datos Originales', color='blue')
    plt.title(f'{os.path.splitext(x)[0]} - {y}')
    plt.xlabel('Índice de Datos')
    plt.ylabel('Valor Normalizado')
    plt.legend()
    plt.grid(True)

    # Guardar la segunda figura en un BytesIO
    buf2 = BytesIO()
    plt.savefig(buf2, format='png')
    plt.show()
    plt.close()
    # Insertar la segunda imagen del gráfico
    slide = ppt.slides.add_slide(ppt.slide_layouts[5])
    left1, top1 = 0, 0  # Ajusta la posición de la primera imagen
    pic2 = slide.shapes.add_picture(buf2, left1, top1, width=ppt.slide_width, height=ppt.slide_height)

    # Crear una figura y un solo eje y
    fig, ax = plt.subplots(figsize=(12, 6))
    # Gráfico de Datos Originales y Predicciones
    ax.plot(data['valor_normalizado'], label='Datos Originales', color='blue', linestyle = '--', linewidth = 0.8)
    ax.plot(predicciones, label='Predicciones', color='black')
    ax.scatter(anomalias.index, anomalias['x_test'], color='red', label='Anomalías', marker='o', s=20)
    ax.set_title('Datos de tráfico vs. Predicciones (con Anomalías)', fontweight = 'bold')
    ax.set_xlabel('Datos de Testeo')
    ax.set_ylabel('Valor Normalizado', color='black')  # Eje izquierdo
    ax.legend(loc='upper left')
    # Crear un segundo conjunto de etiquetas en el eje y para la pérdida y el umbral
    ax2 = ax.twinx()
    ax2.plot(df_test.index, df_test.error2, label='Pérdida', color='red', linestyle='--', linewidth = 1.5)
    ax2.set_ylabel('Pérdida y Umbral', color='black')  # Eje derecho
    ax2.legend(loc='upper right')
    y2_min = 0
    y2_max = max(df_test.error2)*4
    # Establecer los límites en el eje y del segundo conjunto de datos
    ax2.set_ylim(y2_min, y2_max)  # Reemplaza y2_min y y2_max con los valores deseados
    # Ajustar el espaciado entre los gráficos
    plt.tight_layout()
    # Mostrar el gráfico combinado

    # Guardar la figura en un BytesIO para luego insertarla en la diapositiva
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.show()
    plt.close()
    # Crear una nueva diapositiva
    slide = ppt.slides.add_slide(ppt.slide_layouts[5])  # 5 es el diseño de título y contenido

    # Insertar la primera imagen del gráfico
    left1, top1 = 0, 0  # Ajusta la posición de la primera imagen
    pic1 = slide.shapes.add_picture(buf, left1, top1, width=ppt.slide_width, height=ppt.slide_height)

In [ ]:
# Guardar la presentación
ppt.save(r'C:\Users\colazabal\Desktop\Nueva carpeta\ppt.pptx')


In [ ]:
for x in os.listdir(ruta):
    if x.endswith('xlsx'):
        print(x)
        wd = os.path.join(ruta,x)
        data = pd.read_excel(wd)
        for y in data.columns[1:]:
            empresa = y
            test_size = 0.3 # longitud del testeo
            valor = 1 - test_size
            valor1 = valor*len(data[empresa].dropna())
            valor1 = int(valor1) #longitud del la parte de entrenamiento
            TIME_STEPS = 30 #valor que se puede variar en el tiempo
            epochs = 100
            batch_size = 32
            
            #PREPROCESAMIENTO DE LOS DATOS
            data['valor_normalizado'] = data[empresa].dropna().reset_index().drop(columns='index')
            scaler = StandardScaler()
            scaler = scaler.fit(data[['valor_normalizado']])
            data['valor_normalizado'] = scaler.transform(data[['valor_normalizado']])

            def create_dataset(X, y, time_steps=1):
                Xs, ys = [], []
                for i in range(len(X) - time_steps):
                    v = X.iloc[i:(i + time_steps)].values
                    Xs.append(v)        
                    ys.append(y.iloc[i + time_steps])
                return np.array(Xs), np.array(ys)

            X_data, y_data = create_dataset(data[['valor_normalizado']], data.valor_normalizado, TIME_STEPS)

            #predicciones
            predictions = model.predict(X_data)
            predicciones1 = pd.DataFrame(predictions.flat[0:TIME_STEPS])

            predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
            predicciones = pd.concat((predicciones1, predicciones2),axis=0)

            predicciones.reset_index(drop=True, inplace=True)

            #calculo del error de reconstrucción
            df_test = pd.DataFrame()
            df_test['x_test'] = data['valor_normalizado']
            df_test['predic'] = predicciones
            df_test['error'] = df_test['x_test'] - df_test['predic']
            df_test['error2'] = df_test['error']**2

            #cálculo del threshold
            mu = np.mean(df_test['error2'])  # Media
            sigma = np.std(df_test['error2'])  # Desviación estándar
            alpha = 0.05  # Nivel de significancia del 5%
            z_score = stats.norm.ppf(1 - alpha/2) 
            threshold = mu - (z_score * (sigma / np.sqrt(len(df_test['error2'])))) #Umbral gaussiano

            anomalias = df_test[df_test['error2'] > threshold] #creación de la variable anomalias que contiene a aquellos errores mayores al umbral

            # Crear una figura y un solo eje y
            fig, ax = plt.subplots(figsize=(12, 6))
            # Gráfico de Datos Originales y Predicciones
            ax.plot(data['valor_normalizado'], label='Datos Originales', color='blue', linestyle = '--', linewidth = 0.8)
            ax.plot(predicciones, label='Predicciones', color='black')
            ax.scatter(anomalias.index, anomalias['x_test'], color='red', label='Anomalías', marker='o', s=20)
            ax.set_title('Datos de tráfico vs. Predicciones (con Anomalías)', fontweight = 'bold')
            ax.set_xlabel('Datos de Testeo')
            ax.set_ylabel('Valor Normalizado', color='black')  # Eje izquierdo
            ax.legend(loc='upper left')
            
            # Crear un segundo conjunto de etiquetas en el eje y para la pérdida y el umbral
            ax2 = ax.twinx()
            ax2.plot(df_test.index, df_test.error2, label='Pérdida', color='red', linestyle='--', linewidth = 1.5)
            ax2.set_ylabel('Pérdida y Umbral', color='black')  # Eje derecho
            ax2.legend(loc='upper right')
            y2_min = 0
            y2_max = max(df_test.error2)*2
            
            # Establecer los límites en el eje y del segundo conjunto de datos
            ax2.set_ylim(y2_min, y2_max)  # Reemplaza y2_min y y2_max con los valores deseados
            # Ajustar el espaciado entre los gráficos
            plt.tight_layout()
            # Mostrar el gráfico combinado
            plt.savefig(os.path.join(ruta2,f'{x}-{y}.png'))
            plt.show()

In [ ]:

empresa = 'claro'


test_size = 0.3 # longitud del testeo
valor = 1 - test_size
valor1 = valor*400
valor1 = int(valor1) #longitud del la parte de entrenamiento
TIME_STEPS = 30 #valor que se puede variar en el tiempo
epochs = 100 
batch_size = 32

#PREPROCESAMIENTO DE LOS DATOS

data['valor_normalizado'] = data[empresa]
train, test = train_test_split(data['valor_normalizado'], test_size=test_size, shuffle = False)
train = pd.DataFrame(train)
test= pd.DataFrame(test)
scaler = StandardScaler()
scaler = scaler.fit(train[['valor_normalizado']])
train['valor_normalizado'] = scaler.transform(train[['valor_normalizado']])
test['valor_normalizado'] = scaler.transform(test[['valor_normalizado']])

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

X_train, y_train = create_dataset(train[['valor_normalizado']], train.valor_normalizado, TIME_STEPS)
X_test, y_test = create_dataset(test[['valor_normalizado']], test.valor_normalizado, TIME_STEPS)

#CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(X_train.shape[1], activation='linear')])

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_data=(X_train, X_train), verbose=0) #ENTRENAMIENTO
mse = model.evaluate(X_train, X_train, verbose=0)
loss = history.history['loss']
val_loss = history.history['val_loss']

# desprocesamiento de los datos de testeo
a = pd.DataFrame(X_test[0].flat)
b = pd.DataFrame(y_test)
datos_testeo = pd.concat((a,b)).reset_index(drop=True)

#predicciones
predictions = model.predict(X_test)
predicciones1 = pd.DataFrame(predictions[0].flat)
predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
predicciones = pd.concat((predicciones1, predicciones2))
predicciones.reset_index(drop=True, inplace=True)
datos_testeo.reset_index(drop=True, inplace=True)

#calculo del error de reconstrucción
df_test = pd.DataFrame()
df_test['x_test'] = datos_testeo
df_test['predic'] = predicciones
df_test['error'] = df_test['x_test'] - df_test['predic']
df_test['error2'] = df_test['error']**2

#cálculo del threshold
mu = np.mean(df_test['error2'])  # Media
sigma = np.std(df_test['error2'])  # Desviación estándar
alpha = 0.05  # Nivel de significancia del 5%
z_score = stats.norm.ppf(1 - alpha/2)  # Utilizando la función de percentil inverso de la distribución normal
threshold = mu - (z_score * (sigma / np.sqrt(len(df_test['error2'])))) #Umbral gaussiano

anomalias = df_test[df_test['error2'] > threshold] #creación de la variable anomalias que contiene a aquellos errores mayores al umbral


def objective(trial):
    # Definir los espacios de búsqueda para los hiperparámetros
    num_units1 = trial.suggest_int('num_units1', 64, 128)
    num_units2 = trial.suggest_int('num_units2', 32, 64)
    num_units3 = trial.suggest_int('num_units3', 8, 16)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    # Crear un modelo Autoencoder
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1])),
        tf.keras.layers.Dense(num_units1, activation='relu'),
        tf.keras.layers.Dense(num_units2, activation='relu'),
        tf.keras.layers.Dense(num_units3, activation='relu'),
        tf.keras.layers.Dense(num_units2, activation='relu'),
        tf.keras.layers.Dense(num_units1, activation='relu'),
        tf.keras.layers.Dense(X_train.shape[1], activation='linear')])
    # Compilar el modelo con los hiperparámetros seleccionados
    model.compile(optimizer='adam', loss='mean_squared_error')  # Corregido

    # Entrenar el modelo con los datos de entrenamiento y prueba
    history = model.fit(X_train, X_train, epochs=100, batch_size=batch_size, validation_data=(X_train, X_train), verbose=0)

    # Calcular la métrica de interés (puede ser la precisión, el AUC, etc.)
    # En este ejemplo, usaremos la pérdida en los datos de prueba
    loss = model.evaluate(X_train, X_train)  # Corregido

    return loss

# Crear el estudio de Optuna y ejecutar la minimizacion de la pérdida
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Obtener los mejores hiperparámetros encontrados
best_params = study.best_params

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros encontrados:", best_params)

#UTILIZAMOS EL MODELO CON LOS NUEVOS PARÁMETROS, Y BATCHSIZE
#CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1])),
    tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
    tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
    tf.keras.layers.Dense(best_params['num_units3'], activation='relu'),
    tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
    tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
    tf.keras.layers.Dense(X_train.shape[1], activation='linear')])

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, X_train, epochs=50, batch_size=best_params['batch_size'], validation_data=(X_train, X_train), verbose=0)
mse = model.evaluate(X_train, X_train, verbose=0)

a = pd.DataFrame(X_test[0].flat)
b = pd.DataFrame(y_test)
datos_testeo = pd.concat((a,b)).reset_index(drop=True)

predictions = model.predict(X_test)
predicciones1 = pd.DataFrame(predictions[0].flat)
predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
predicciones = pd.concat((predicciones1, predicciones2))

predicciones.reset_index(drop=True, inplace=True)
datos_testeo.reset_index(drop=True, inplace=True)

loss = history.history['loss']
val_loss = history.history['val_loss']

df_test = pd.DataFrame()
df_test['x_test'] = datos_testeo
df_test['predic'] = predicciones
df_test['error'] = df_test['x_test'] - df_test['predic']
df_test['error2'] = df_test['error']**2

anomalias = df_test[df_test['error2'] > threshold] #ANOMALIA DE ACUERDO A LO QUE SEA MAYOR AL THRESHOLD

# Crear una figura y un solo eje y
fig, ax = plt.subplots(figsize=(12, 6))
# Gráfico de Datos Originales y Predicciones
ax.plot(datos_testeo, label='Datos Originales', color='blue', linestyle = '--', linewidth = 0.8)
ax.plot(predicciones, label='Predicciones', color='black')
ax.scatter(anomalias.index, anomalias['x_test'], color='red', label='Anomalías', marker='o', s=20)
ax.set_title('Datos de tráfico vs. Predicciones (con Anomalías)', fontweight = 'bold')
ax.set_xlabel('Datos de Testeo')
ax.set_ylabel('Valor Normalizado', color='black')  # Eje izquierdo
ax.legend(loc='upper left')

# Crear un segundo conjunto de etiquetas en el eje y para la pérdida y el umbral
ax2 = ax.twinx()
ax2.plot(df_test.index, df_test.error2, label='Pérdida', color='red', linestyle='--', linewidth = 1.5)
ax2.set_ylabel('Pérdida y Umbral', color='black')  # Eje derecho
ax2.legend(loc='upper right')
y2_min = 0
y2_max = max(df_test.error2)*2

# Establecer los límites en el eje y del segundo conjunto de datos
ax2.set_ylim(y2_min, y2_max)  # Reemplaza y2_min y y2_max con los valores deseados

# Ajustar el espaciado entre los gráficos
plt.tight_layout()

# Mostrar el gráfico combinado
plt.show()